In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_services_and_industries(url):
    chrome_options = Options()

    # Headless nhưng chống detect
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--window-size=1920,1080")

    # Fake user agent
    chrome_options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )

    # Fake Accept-Language
    chrome_options.add_argument("--accept-language=en-US,en;q=0.9")

    # Chromium binary & driver path
    chrome_options.binary_location = "/usr/bin/chromium-browser"
    service = Service("/usr/lib/chromium-browser/chromedriver")

    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Remove webdriver flag
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                    get: () => undefined
                });
            """
        }
    )

    # Fake navigator.plugins & languages
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                Object.defineProperty(navigator, 'plugins', {
                    get: () => [1, 2, 3]
                });

                Object.defineProperty(navigator, 'languages', {
                    get: () => ['en-US', 'en']
                });
            """
        }
    )

    # Fake WebGL
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                const getParameter = WebGLRenderingContext.prototype.getParameter;
                WebGLRenderingContext.prototype.getParameter = function(param) {
                    if (param === 37445) return 'Intel Inc.';
                    if (param === 37446) return 'Intel Iris OpenGL Engine';
                    return getParameter(param);
                };
            """
        }
    )

    try:
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        return soup
    finally:
        driver.quit()



if __name__ == "__main__":
    data = extract_services_and_industries("https://clutch.co/profile/betterworld-technology")
    print(data)


<html lang="en"><head><script async="" src="https://cdn.us.heap-api.com/v5/heapjs-static/5.3.4/core/heap.js" type="text/javascript"></script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>BetterWorld Technology Reviews (26), Pricing, Services &amp; Verified Ratings</title>
<meta content="BetterWorld Technology is your organization’s trusted IT Partner.
 
We focus on 3 business lines: Managed Services, Cybersecurity, and Professional" name="description"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/><link href="https://clutch.co/profile/betterworld-technology" rel="canonical"/>
<meta content="BetterWorld Technology" property="og:title"/>
<meta content="BetterWorld Technology is your organization&amp;rsquo;s trusted IT Partner.
&amp;nbsp;
We focus 

In [9]:
data


<html lang="en"><head><script async="" src="https://cdn.us.heap-api.com/v5/heapjs-static/5.3.4/core/heap.js" type="text/javascript"></script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>BetterWorld Technology Reviews (26), Pricing, Services &amp; Verified Ratings</title>
<meta content="BetterWorld Technology is your organization’s trusted IT Partner.
 
We focus on 3 business lines: Managed Services, Cybersecurity, and Professional" name="description"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/><link href="https://clutch.co/profile/betterworld-technology" rel="canonical"/>
<meta content="BetterWorld Technology" property="og:title"/>
<meta content="BetterWorld Technology is your organization&amp;rsquo;s trusted IT Partner.
&amp;nbsp;
We focus 

In [5]:
services = []
services_section = data.find_all('li', class_ = 'chart-legend--item')
print('---------------->')
print('services_section :', services_section)
print('---------------->')
for item in services_section:
    a = item.find("a", class_="chart-legend--item-link")
    if a:
        services.append(a.get_text(strip=True))
services

---------------->
services_section : [<li class="chart-legend--item" data-id="1" id="legend_1"><h3><a aria-label="See More IT Managed Services Providers" class="chart-legend--item-link" href="https://clutch.co/it-services/msp" target="_blank" title="See More IT Managed Services Providers">
                    IT Managed Services
                </a></h3><span>50%</span></li>, <li class="chart-legend--item" data-id="2" id="legend_2"><h3><a aria-label="See More Cybersecurity Providers" class="chart-legend--item-link" href="https://clutch.co/it-services/cybersecurity" target="_blank" title="See More Cybersecurity Providers">
                    Cybersecurity
                </a></h3><span>30%</span></li>, <li class="chart-legend--item" data-id="3" id="legend_3"><h3><a aria-label="See More IT Strategy Consulting Providers" class="chart-legend--item-link" href="https://clutch.co/it-services" target="_blank" title="See More IT Strategy Consulting Providers">
                    IT Strategy C

['IT Managed Services', 'Cybersecurity', 'IT Strategy Consulting']